<a href="https://colab.research.google.com/github/aRod209/pytorch-for-deep-learning/blob/main/exercises/05_pytorch_going_modular_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
%%writefile get_data.py
"""
Downloads a zipfile of data consisting of pizza, steak, and sushi images.
The zipfile is unzipped and the images are stored in an image path directory.
The Zipfile is then removed from the directory.
"""
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'
image_zip_path = data_path / 'pizza_steak_sushi.zip'

# If the image folder doesn't exist, download it and prepare it.
if image_path.is_dir():
  print(f'{image_path} directory exists.')
else:
  print(f'Did not find {image_path} directory, creating one...')
  image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, and sushi data
with open(image_zip_path, 'wb') as f:
  request  = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
  print('Downloading pizza, steak, and sushi data...')
  f.write(request.content)

# Unzip pizza, steak, and sushi data
with zipfile.ZipFile(image_zip_path, 'r') as zip_ref:
  print('Unzipping pizza, steak, and sushi data...')
  zip_ref.extractall(image_path)

# Remove zip file
os.remove(image_zip_path)

Writing get_data.py


In [2]:
# Example running of get_data.py
!python get_data.py

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, and sushi data...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [3]:
%%writefile data_setup.py
"""
Sets up the data that is needed for model training and testing.
"""
# Standard library imports
import os
from pathlib import Path

# Third-party imports
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision


BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

def create_data_directories(parent_dir:Path) -> tuple:
  """Creates Paths for training data and testing data.

  Uses a parent directory to create and returns Path objects
  for the training data directory and the testing data directory.

  Args:
  parent_dir: A parent Path for the image data.

  Returns:
  A tuple of (Path, Path) representing the path directories of the
  training data and testing data respectively.

  Example usage:
  train_dir, test_dir = create_data_directories(image_path="path/to/images")
  """
  train_dir = parent_dir / 'train'
  test_dir = parent_dir / 'test'
  return train_dir, test_dir

def create_data_transform() -> transforms.Compose:
  """Creates a DataTransform.

  Creates a Compose object that will apply two transforms.
  First the Compose object will resize an image to 64X64 pixels and
  then transform the image to a PyTorch tensor.

  Returns:
  A Compose object that applies two transforms that resizes an image
  and turns the image into a tensor.

  Example usage:
  data_transform = create_data_transform
  """
  data_transform = transforms.Compose([
      transforms.Resize(size=(64, 64)),
      transforms.ToTensor()])
  return data_transform

def create_datasets(train_dir: Path,
                    test_dir: Path,
                    transform: transforms.Compose) -> tuple:
  """Creates training and testing Datasets.

  Takes in Paths, for the training data directory and testing data directory,
  and a Transform to build a tuple of ImageFolder Datasets.

  Args:
  train_dir: Path to training data directory.
  test_dir: Path to testing data directory.
  transform: torchvision transforms to perform on training and testing data.

  Returns:
  A tuple of (ImageFolder, ImageFolder).

  Example usage:
  train_data, test_data = create_datasets(train_dir:train_path,
    test_dir=test_path,
    transform=data_transform)
  """
  train_data = datasets.ImageFolder(root=train_dir, transform=transform)
  test_data = datasets.ImageFolder(root=test_dir, transform=transform)
  return train_data, test_data

def create_dataloaders(train_data: str,
                       test_data: str,
                       batch_size:int=BATCH_SIZE,
                       num_workers: int=NUM_WORKERS) -> tuple:
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns them into
  PyTorch Datasts and then into PyTorch DataLoaders.

  Args:
  train_data: Training data.
  test_data: Test data.
  batch_size: Number of samples per batch in each of te DataLoaders.
  num_workers: An integer of number of workers per DataLoader.

  Returns:
  A tuple of (DataLoader, DataLoader).

  Example usage:
  train_dataloader, test_dataloader, class_names = create_dataloader(
    train_data=train_data,
    test_data=test_data,
    batch_size=32,
    num_workers=4
  )
  """

  # Turn datasets into DataLoaders
  train_dataloader = DataLoader(
      dataset=train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True
  )

  test_dataloader = DataLoader(
      dataset=test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True
  )

  return train_dataloader, test_dataloader

Writing data_setup.py


In [4]:
%%writefile model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model from the
CNN Explainer website.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN Explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    """The forward pass.

    First computes the first convulational block, followed by the 2nd
    convulutional block followed by the classifer.

    Args:
     x: The tensor to compute in the forward pass.

    Returns:
     A tensor that was computed in the forward pass.
    """
    return self.classifier(self.conv_block_2(self.conv_block_1(x)))


Writing model_builder.py


In [5]:
%%writefile engine.py
"""
Contains functions for training and testing PyTorch model.
"""
# Standard library imports
from typing import Dict, List, Tuple

# Third-party imports
import torch

from tqdm.auto import tqdm

# Application-specific imports

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyiTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward,
  pass, loss calculation, optimilzer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu")

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train,accuracy). For example:

  (0.1111, 0.2152)
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader batches
  for batch, (X, y) in enumerate(dataloader):
    # Send data to target device
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_logits = model(X)

    # 2. Calculate the loss
    loss = loss_fn(y_logits, y)
    train_loss += loss.item()

    # 3. Optimzer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate and accumulate accuracy metric across all batches
    y_preds = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
    train_acc += (y_preds == y).sum().item()/len(y_logits)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)

  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a teting dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0222, 0.08988)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_logits = model(X)

      # 2. Calculate and accumulate loss
      loss = loss_fn(y_logits, y)
      test_loss += loss.item()

      # Calculate and accumulate accuracy
      test_preds = y_logits.argmax(dim=1)
      test_acc = (test_preds == y).sum().item()/len(y_logits)

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)

  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch model through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  int the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instacne for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate lass on both datasets.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {
    "train_loss": [],
    "train_acc": [],
    "test_loss": [],
    "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)

    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    # Print out what's happening
    print(
        f"Epoch: {epoch+1} | "
        f"train_loss: {train_loss:.4f} |"
        f"train_acc: {train_acc:.4f} |"
        f"test_loss: {test_loss:.4f} |"
        f"test_acc: {test_acc:.4f}"
    )

    # Update results dictionary
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss)
    results['test_acc'].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Writing engine.py


In [6]:
%%writefile utils.py
"""
File containing various utility functions for PyTorch model training.
"""
# Standard library imports
from pathlib import Path

# Third-party imports
import torch

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth"  or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
    target_dir="models",
    model_name="tinyvgg_model.pth")
  """
  # Create target directory
  if target_dir:
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                          exist_ok=True)

  # Create model save path
  assert model_name.endswith('.pth') or model_name.endswith('.pt'), "model_name should end with '.pt' or '.pth'"

  if target_dir:
    model_save_path = target_dir_path / model_name
  else:
    model_save_path = model_name

  # Save the model state_dict()
  print(f'[INFO] Saving model to: {model_save_path}')
  torch.save(obj=model.state_dict(),
             f=model_save_path)


Writing utils.py


In [7]:
%%writefile train.py
"""
Trains the data.
"""
# Standard library imports
import argparse
import os

from pathlib import Path
from timeit import default_timer as timer

# Third-party imports
import torch

# Application-specific imports
import data_setup
import engine
import get_data
import model_builder
import utils

# Create the argument parser
# Create the argument parser
parser = argparse.ArgumentParser(description="Training script for a neural network.")

# Add arguments
parser.add_argument('--num_epochs', type=int, default=5, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=32, help='Batch size for training.')
parser.add_argument('--hidden_units', type=int, default=10, help='Number of hidden units in the neural network.')
parser.add_argument('--learning_rate', type=float, default=0.001, help='Learning rate for the optimizer.')
parser.add_argument('--model_name', type=str, default='model_0', help='A filename for the saved model.')
parser.add_argument('--model_dir', type=str, default='', help='The directory to save the model to.')

# Parse the arguments
args = parser.parse_args()

# Use the arguments
print(f'Training with settings: epochs={args.num_epochs}, batch size={args.batch_size}, '
      f'hidden units={args.hidden_units}, learning rate={args.learning_rate}')

# Setup hyperparameters
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
INPUT_SHAPE = 3
LEARNING_RATE = args.learning_rate
NUM_EPOCHS = args.num_epochs
NUM_WORKERS = os.cpu_count()
IMAGE_DATA_PATH = 'data/pizza_steak_sushi'
MODEL_NAME = args.model_name
MODEL_DIR = args.model_dir

# Setup device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Setup directories
parent_dir = Path(IMAGE_DATA_PATH)
train_dir, test_dir = data_setup.create_data_directories(parent_dir)

# Create transforms
data_transform = data_setup.create_data_transform()

# Create data sets
train_data, test_data = data_setup.create_datasets(train_dir=train_dir,
                                                   test_dir=test_dir,
                                                   transform=data_transform)

# Create data loaders
train_dataloader, test_dataloader = data_setup.create_dataloaders(train_data=train_data,
                                                                  test_data=test_data,
                                                                  batch_size=BATCH_SIZE,
                                                                  num_workers=NUM_WORKERS)


# Create model
model = model_builder.TinyVGG(input_shape=INPUT_SHAPE,
                              hidden_units=HIDDEN_UNITS,
                              output_shape=len(train_data.classes)).to(device)

# Setup loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=0.001)

# Start the timer
start_time = timer()

# Setup training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# end the timer ad print out how long it took
end_time = timer()

print(f'[INFO] Total training time: {end_time-start_time:.3f} seconds')

# Save the model to a file
utils.save_model(model=model,
                 target_dir=MODEL_DIR,
                 model_name=MODEL_NAME)

Writing train.py


In [8]:
# Example running of train.py
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003 --model_name 'tinyvgg_model.pt'

data/pizza_steak_sushi directory exists.
Unzipping pizza, steak, and sushi data...
Training with settings: epochs=5, batch size=128, hidden units=128, learning rate=0.0003
  0% 0/5 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch: 1 | train_loss: 1.1288 |train_acc: 0.3649 |test_loss: 1.0922 |test_acc: 0.4800
 20% 1/5 [00:03<00:13,  3.49s/it]Epoch: 2 | train_loss: 1.0932 |train_acc: 0.3579 |test_loss: 1.0740 |test_acc: 0.4267
 40% 2/5 [00:04<00:06,  2.17s/it]Epoch: 3 | train_loss: 1.0601 |train_acc: 0.4077 |test_loss: 1.0665 |test_acc: 0.3867
 60% 3/5 [00:05<00:03,  1.73s/it]Epoch: 4 | train_loss: 0.9871 |train_acc: 0.5921 |test_loss: 1.0897 |test_acc: 0.4267
 80% 4/5 [00:07<00:01,  1.52s/it]Epoch: 5 | train_loss: 0.9681 |train_acc: 0.5486 |test_loss: 1.0370 |test_acc: 0.4133
100% 5/5 

## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [ ]:
%%writefile predict.py
"""
Makes prediction on a custom image given a file path with a saved model
"""
# Standard library imports
import argparse

# Create argument parser
parser = argparse.ArgumentParser(description='Prediction script using a saved model on a given file path for an image.')

# Add arguments
parser.add_argument('--image', type=str, default='', help='File path of target image.', required=False)
parser.add_argument('--saved_model', type=str, default='', help='File path to the saved PyTorch model.', required=True)

# Parse the arguments
args = parser.parse_args()

print(f'Predicting with settings: image={args.image}, saved_model={args.saved_model}')

# Setup hyperparameters
image = args.image
saved_model = args.saved_model

# Load image

In [ ]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

python3: can't open file '/content/predict.py': [Errno 2] No such file or directory
